In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn import cross_validation, metrics   #Additional     scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search
import datetime
from math import*
from datetime import date
from sklearn import preprocessing
import geohash
from sklearn.model_selection import train_test_split
import datetime
from sklearn.utils import shuffle  
import warnings
warnings.filterwarnings("ignore")
#geohash 3-11
#round 7-9

/Users/destiny/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/destiny/anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
################geohash######################
def get_geohash(x,precision_i):
    return geohash.encode(longitude=float(x.longitude),latitude=float(x.latitude),precision=precision_i)

def get_geohash_round1(geohash_u):
    return geohash.neighbors(geohash_u)[0]
def get_geohash_round2(geohash_u):
    return geohash.neighbors(geohash_u)[1]
def get_geohash_round3(geohash_u):
    return geohash.neighbors(geohash_u)[2]
def get_geohash_round4(geohash_u):
    return geohash.neighbors(geohash_u)[3]
def get_geohash_round5(geohash_u):
    return geohash.neighbors(geohash_u)[4]
def get_geohash_round6(geohash_u):
    return geohash.neighbors(geohash_u)[5]
def get_geohash_round7(geohash_u):
    return geohash.neighbors(geohash_u)[6]
def get_geohash_round8(geohash_u):
    return geohash.neighbors(geohash_u)[7]



In [2]:
shop_info=pd.read_csv("../ccf_first_round_shop_info.csv")
trade_info=pd.read_csv("../ccf_first_round_user_shop_behavior.csv")
all_info=pd.merge(trade_info,shop_info,on='shop_id',how='left')
test_info=pd.read_csv("../evaluation_public.csv")
all_info.rename(columns={'longitude_x': 'longitude', 'latitude_x': 'latitude'}, inplace=True)
train_wifi=pd.read_csv("../dealFile/train_index_wifi.csv")
test_wifi=pd.read_csv("../dealFile/test_index_wifi.csv")
train_con = pd.concat([all_info,train_wifi],axis=1)
test_con=pd.concat([test_info,test_wifi],axis=1)
for i in range(0,10):
    train_con[['bss'+str(i)]]= train_con[['bss'+str(i)]].fillna('0')
    train_con[['sig'+str(i)]]= train_con[['sig'+str(i)]].fillna('-100')
    test_con[['bss'+str(i)]]= test_con[['bss'+str(i)]].fillna('0')
    test_con[['sig'+str(i)]]= test_con[['sig'+str(i)]].fillna('-100')


shop_info['count'] = 1
shop_g=shop_info.groupby(["mall_id"]).agg('sum').reset_index()


mall_group=shop_g.mall_id
mall_data={}
for value in mall_group.values:
    mall_data[value]=train_con[train_con.mall_id==value]

del trade_info,all_info,test_info

In [3]:
tes=test_con[test_con.row_id=='m_1263']

In [7]:
train_con.count()

user_id        1138015
shop_id        1138015
time_stamp     1138015
longitude      1138015
latitude       1138015
wifi_infos     1138015
category_id    1138015
longitude_y    1138015
latitude_y     1138015
price          1138015
mall_id        1138015
count          1138015
connect        1138015
bss0           1138015
sig0           1138015
bss1           1138015
sig1           1138015
bss2           1138015
sig2           1138015
bss3           1138015
sig3           1138015
bss4           1138015
sig4           1138015
bss5           1138015
sig5           1138015
bss6           1138015
sig6           1138015
bss7           1138015
sig7           1138015
bss8           1138015
sig8           1138015
bss9           1138015
sig9           1138015
dtype: int64

In [6]:
def processTime(train,test):
    ##########time_stamp##########
    da=datetime.datetime.now()
    l=da
    train['day_of_week']=train.time_stamp.astype('str').apply(lambda x:date(int(x[0:4]),int(x[5:7]),int(x[8:10])).weekday()+1)
    train['day_of_month']=train.time_stamp.astype('str').apply(lambda x:date(int(x[0:4]),int(x[5:7]),int(x[8:10])).day)
    train['hour']=train.time_stamp.astype('str').apply(lambda x:int(x[11:13]))
    train['minute']=train.time_stamp.astype('str').apply(lambda x:int(x[14:16]))


    #test
    test['day_of_week']=test.time_stamp.astype('str').apply(lambda x:date(int(x[0:4]),int(x[5:7]),int(x[8:10])).weekday()+1)
    test['day_of_month']=train.time_stamp.astype('str').apply(lambda x:date(int(x[0:4]),int(x[5:7]),int(x[8:10])).day)
    test['hour']=test.time_stamp.astype('str').apply(lambda x:int(x[11:13]))
    test['minute']=test.time_stamp.astype('str').apply(lambda x:int(x[14:16]))

    print "process Time"
    return train,test

In [12]:
def processLoc(train,test):
    for i in range(7,13):
        train['geohash_'+str(i)]=train.apply(get_geohash,axis=1,args=(i,))
        test['geohash_'+str(i)]=test.apply(get_geohash,axis=1,args=(i,))
    train['geohash_91']=train.geohash_9.apply(get_geohash_round1)
    train['geohash_92']=train.geohash_9.apply(get_geohash_round2)
    train['geohash_93']=train.geohash_9.apply(get_geohash_round3)
    train['geohash_94']=train.geohash_9.apply(get_geohash_round4)
    train['geohash_95']=train.geohash_9.apply(get_geohash_round5)
    train['geohash_96']=train.geohash_9.apply(get_geohash_round6)
    train['geohash_97']=train.geohash_9.apply(get_geohash_round7)
    train['geohash_98']=train.geohash_9.apply(get_geohash_round8)
    
    test['geohash_91']=test.geohash_9.apply(get_geohash_round1)
    test['geohash_92']=test.geohash_9.apply(get_geohash_round2)
    test['geohash_93']=test.geohash_9.apply(get_geohash_round3)
    test['geohash_94']=test.geohash_9.apply(get_geohash_round4)
    test['geohash_95']=test.geohash_9.apply(get_geohash_round5)
    test['geohash_96']=test.geohash_9.apply(get_geohash_round6)
    test['geohash_97']=test.geohash_9.apply(get_geohash_round7)
    test['geohash_98']=test.geohash_9.apply(get_geohash_round8)
    
    train['geohash_101']=train.geohash_10.apply(get_geohash_round1)
    train['geohash_102']=train.geohash_10.apply(get_geohash_round2)
    train['geohash_103']=train.geohash_10.apply(get_geohash_round3)
    train['geohash_104']=train.geohash_10.apply(get_geohash_round4)
    train['geohash_105']=train.geohash_10.apply(get_geohash_round5)
    train['geohash_106']=train.geohash_10.apply(get_geohash_round6)
    train['geohash_107']=train.geohash_10.apply(get_geohash_round7)
    train['geohash_108']=train.geohash_10.apply(get_geohash_round8)
    
    test['geohash_101']=test.geohash_10.apply(get_geohash_round1)
    test['geohash_102']=test.geohash_10.apply(get_geohash_round2)
    test['geohash_103']=test.geohash_10.apply(get_geohash_round3)
    test['geohash_104']=test.geohash_10.apply(get_geohash_round4)
    test['geohash_105']=test.geohash_10.apply(get_geohash_round5)
    test['geohash_106']=test.geohash_10.apply(get_geohash_round6)
    test['geohash_107']=test.geohash_10.apply(get_geohash_round7)
    test['geohash_108']=test.geohash_10.apply(get_geohash_round8)
    
    for i in [7,8,11,12]:    
        geo_lbl_1=preprocessing.LabelEncoder()
        geo_lbl_1.fit(list(set(train['geohash_'+str(i)].values)|set(test['geohash_'+str(i)].values)))
        train['geohash_'+str(i)]=geo_lbl_1.transform(train['geohash_'+str(i)].values)
        test['geohash_'+str(i)]=geo_lbl_1.transform(test['geohash_'+str(i)].values)
        
    geo_lbl=preprocessing.LabelEncoder()
    geo_lbl.fit(list(set(train['geohash_9'].values)|set(test['geohash_9'].values)|\
    set(train['geohash_91'])|set(test['geohash_91'])|\
    set(train['geohash_92'])|set(test['geohash_92'])|\
    set(train['geohash_93'])|set(test['geohash_93'])|\
    set(train['geohash_94'])|set(test['geohash_94'])|\
    set(train['geohash_95'])|set(test['geohash_95'])|\
    set(train['geohash_96'])|set(test['geohash_96'])|\
    set(train['geohash_97'])|set(test['geohash_97'])|\
    set(train['geohash_98'])|set(test['geohash_98'])))

    train['geohash_9']=geo_lbl.transform(train['geohash_9'].values)
    train['geohash_91']=geo_lbl.transform(train['geohash_91'].values)
    train['geohash_92']=geo_lbl.transform(train['geohash_92'].values)
    train['geohash_93']=geo_lbl.transform(train['geohash_93'].values)
    train['geohash_94']=geo_lbl.transform(train['geohash_94'].values)
    train['geohash_95']=geo_lbl.transform(train['geohash_95'].values)
    train['geohash_96']=geo_lbl.transform(train['geohash_96'].values)
    train['geohash_97']=geo_lbl.transform(train['geohash_97'].values)
    train['geohash_98']=geo_lbl.transform(train['geohash_98'].values)

    test['geohash_9']=geo_lbl.transform(test['geohash_9'].values)
    test['geohash_91']=geo_lbl.transform(test['geohash_91'].values)
    test['geohash_92']=geo_lbl.transform(test['geohash_92'].values)
    test['geohash_93']=geo_lbl.transform(test['geohash_93'].values)
    test['geohash_94']=geo_lbl.transform(test['geohash_94'].values)
    test['geohash_95']=geo_lbl.transform(test['geohash_95'].values)
    test['geohash_96']=geo_lbl.transform(test['geohash_96'].values)
    test['geohash_97']=geo_lbl.transform(test['geohash_97'].values)
    test['geohash_98']=geo_lbl.transform(test['geohash_98'].values)
    
    geo_lbl=preprocessing.LabelEncoder()
    geo_lbl.fit(list(set(train['geohash_10'].values)|set(test['geohash_10'].values)|\
    set(train['geohash_101'])|set(test['geohash_101'])|\
    set(train['geohash_102'])|set(test['geohash_102'])|\
    set(train['geohash_103'])|set(test['geohash_103'])|\
    set(train['geohash_104'])|set(test['geohash_104'])|\
    set(train['geohash_105'])|set(test['geohash_105'])|\
    set(train['geohash_106'])|set(test['geohash_106'])|\
    set(train['geohash_107'])|set(test['geohash_107'])|\
    set(train['geohash_108'])|set(test['geohash_108'])))

    train['geohash_10']=geo_lbl.transform(train['geohash_10'].values)
    train['geohash_101']=geo_lbl.transform(train['geohash_101'].values)
    train['geohash_102']=geo_lbl.transform(train['geohash_102'].values)
    train['geohash_103']=geo_lbl.transform(train['geohash_103'].values)
    train['geohash_104']=geo_lbl.transform(train['geohash_104'].values)
    train['geohash_105']=geo_lbl.transform(train['geohash_105'].values)
    train['geohash_106']=geo_lbl.transform(train['geohash_106'].values)
    train['geohash_107']=geo_lbl.transform(train['geohash_107'].values)
    train['geohash_108']=geo_lbl.transform(train['geohash_108'].values)

    test['geohash_10']=geo_lbl.transform(test['geohash_10'].values)
    test['geohash_101']=geo_lbl.transform(test['geohash_101'].values)
    test['geohash_102']=geo_lbl.transform(test['geohash_102'].values)
    test['geohash_103']=geo_lbl.transform(test['geohash_103'].values)
    test['geohash_104']=geo_lbl.transform(test['geohash_104'].values)
    test['geohash_105']=geo_lbl.transform(test['geohash_105'].values)
    test['geohash_106']=geo_lbl.transform(test['geohash_106'].values)
    test['geohash_107']=geo_lbl.transform(test['geohash_107'].values)
    test['geohash_108']=geo_lbl.transform(test['geohash_108'].values)
    print "process geo"
    return train,test

In [7]:
def proceeWifi(train,test,num=4):
    #test
    lbl=preprocessing.LabelEncoder()
    lbl.fit(list(set(train['connect'].values)|set(train['bss0'].values)|set(train['bss1'].values)|\
    set(train['bss2'].values)|set(train['bss3'].values)|set(train['bss4'].values)|\
    set(train['bss5'].values)|set(train['bss6'].values)|\
    set(train['bss7'].values)|set(train['bss8'].values)|\
    set(train['bss9'].values)|set(test['connect'].values)|\
    set(test['bss0'].values)|set(test['bss1'].values)|\
    set(test['bss2'].values)|set(test['bss3'].values)|set(test['bss4'].values)|\
    set(test['bss5'].values)|set(test['bss6'].values)|\
    set(test['bss7'].values)|set(test['bss8'].values)|set(test['bss9'].values)))
    train['connect']=lbl.transform(train['connect'].values)
    test['connect']=lbl.transform(test['connect'].values)
    train['bss0']=lbl.transform(train['bss0'].values)
    test['bss0']=lbl.transform(test['bss0'].values)
    train['bss1']=lbl.transform(train['bss1'].values)
    test['bss1']=lbl.transform(test['bss1'].values)
    train['bss2']=lbl.transform(train['bss2'].values)
    test['bss2']=lbl.transform(test['bss2'].values)
    train['bss3']=lbl.transform(train['bss3'].values)
    test['bss3']=lbl.transform(test['bss3'].values)
    train['bss4']=lbl.transform(train['bss4'].values)
    test['bss4']=lbl.transform(test['bss4'].values)
    train['bss5']=lbl.transform(train['bss5'].values)
    test['bss5']=lbl.transform(test['bss5'].values)
    train['bss6']=lbl.transform(train['bss6'].values)
    test['bss6']=lbl.transform(test['bss6'].values)
    train['bss7']=lbl.transform(train['bss7'].values)
    test['bss7']=lbl.transform(test['bss7'].values)
    train['bss8']=lbl.transform(train['bss8'].values)
    test['bss8']=lbl.transform(test['bss8'].values)
    train['bss9']=lbl.transform(train['bss9'].values)
    test['bss9']=lbl.transform(test['bss9'].values)
    
    print "process wifi"
    return train,test

In [8]:
def processUser(train,test):
    a=train.user_id.values
    b=test.user_id.values
    user_share=list(set(a).intersection(set(b)))
    train_n_in=~train.user_id.isin(user_share).values
    test_n_in=~test.user_id.isin(user_share).values
    train.loc[train_n_in,'user_id']='0'
    test.loc[test_n_in,'user_id']='0'
    
    
    lbl=preprocessing.LabelEncoder()
    lbl.fit(list(set(train['user_id'].values)|set(test['user_id'].values)))
    train['user_id']=lbl.transform(train['user_id'].values)
    test['user_id']=lbl.transform(test['user_id'].values)
    return train,test

In [9]:
fea_imp=[0  for i in range(0,52)]
s_times=1

In [14]:
def generateData11(mall_data,mall_id):
    print str(s_times) +"  "+str(mall_id)
    print datetime.datetime.now()
    train=mall_data
    test=test_con[test_con.mall_id==mall_id]
    
    shop_dict = {}
    exam_shop = shop_info[shop_info.mall_id == mall_id]
    i1 = 0
    for index, row in exam_shop.iterrows():
        shop_dict[row.shop_id] = i1
        i1 = i1 + 1
    shop_ito_shop = dict((v, k) for k, v in shop_dict.iteritems())
    
    train,test=processUser(train,test)
    train,test=processTime(train,test)
    train,test=processLoc(train,test)
    train,test=proceeWifi(train,test)    

    train=shuffle(train)
    
    label_train=train[['shop_id']]
    label_num=len(shop_info[shop_info.mall_id==mall_id].shop_id.values)
    label_to_train=[]
    label_train=label_train.shop_id.values
    for l in label_train:
        label_to_train.append(shop_dict[l])

    train.drop(['shop_id','time_stamp','category_id', 'longitude_y','latitude_y', 'price', 'mall_id','wifi_infos'],axis=1,inplace=True)
    row_id_list = test.row_id.values
    test.drop(['row_id','time_stamp','mall_id','wifi_infos'],axis=1,inplace=True)
    

        
    
    xg_train=train.as_matrix()
    xg_test=test.as_matrix()
    

    
    gbmclf=lgb.LGBMClassifier(learning_rate=0.03,lambda_l2=0.3,feature_fraction=0.8,min_data_in_leaf=200,min_sum_hessian_in_leaf=0.1,n_estimators=1000,max_depth=7,num_class=label_num,subsample=0.8,boosting_type='gbdt',max_bin=255,min_child_weight=1,objective='multiclass',metric='multi_logloss')
    

    gbmclf.fit(xg_train,label_to_train)
    print gbmclf.score(xg_train,label_to_train)
    
    test_res = gbmclf.predict(xg_test)
    

    with open("res10313.csv",'a+') as f:
        for i in range(len(test_res)):
            f.write(str(row_id_list[i]) + ',' + str(shop_ito_shop[test_res[i]] + '\n'))
    print datetime.datetime.now()
    
    
    
        
    

In [13]:
gcv=generateData11(mall_data['m_1263'],'m_1263')

1  m_1263
2017-10-31 22:04:08.494869
process Time
process geo
process wifi


KeyboardInterrupt: 

In [19]:
gcv.best_params_

{'n_estimators': 800}

In [20]:
gcv.best_score_

0.8394066430183812

In [15]:
gcv.best_score_

0.8329571106094809

In [16]:
gcv.best_params_

{'learning_rate': 0.1}

In [15]:
for i in mall_data.keys():
    generateData11(mall_data[i],i)
    s_times=s_times+1

1  m_1263
2017-10-31 22:07:43.287508
process Time
process geo
process wifi
0.999892507793
2017-10-31 22:12:46.195534
2  m_3517
2017-10-31 22:12:46.307709
process Time
process geo
process wifi
1.0
2017-10-31 22:17:07.925899
3  m_5892
2017-10-31 22:17:08.042062
process Time
process geo
process wifi
1.0
2017-10-31 22:25:13.121910
4  m_3054
2017-10-31 22:25:13.280734
process Time
process geo
process wifi
1.0
2017-10-31 22:30:17.130736
5  m_4168
2017-10-31 22:30:17.243091
process Time
process geo
process wifi
1.0
2017-10-31 22:33:18.513794
6  m_4406
2017-10-31 22:33:18.631073
process Time
process geo
process wifi
1.0
2017-10-31 22:36:01.578525
7  m_2878
2017-10-31 22:36:01.670236
process Time
process geo
process wifi
0.9999031477
2017-10-31 22:41:34.915814
8  m_3112
2017-10-31 22:41:35.096602
process Time
process geo
process wifi
1.0
2017-10-31 22:45:21.414071
9  m_625
2017-10-31 22:45:21.505455
process Time
process geo
process wifi
1.0
2017-10-31 22:49:53.213063
10  m_626
2017-10-31 22:49:

2017-11-01 04:30:12.261524
76  m_6803
2017-11-01 04:30:12.406528
process Time
process geo
process wifi
1.0
2017-11-01 04:31:30.274446
77  m_1089
2017-11-01 04:31:30.367548
process Time
process geo
process wifi
1.0
2017-11-01 04:34:02.302844
78  m_2715
2017-11-01 04:34:02.427895
process Time
process geo
process wifi
1.0
2017-11-01 04:36:19.383485
79  m_5352
2017-11-01 04:36:19.489251
process Time
process geo
process wifi
1.0
2017-11-01 04:39:52.921437
80  m_4548
2017-11-01 04:39:53.026429
process Time
process geo
process wifi
1.0
2017-11-01 04:41:59.194176
81  m_1831
2017-11-01 04:41:59.295171
process Time
process geo
process wifi
0.999899030695
2017-11-01 04:45:05.880467
82  m_3839
2017-11-01 04:45:05.971148
process Time
process geo
process wifi
0.999911789353
2017-11-01 04:55:15.963127
83  m_3871
2017-11-01 04:55:16.144297
process Time
process geo
process wifi
0.99993905412
2017-11-01 05:02:55.557635
84  m_3501
2017-11-01 05:02:55.740155
process Time
process geo
process wifi
1.0
2017-

In [42]:
for i in mall_data.keys():
    generateData11(mall_data[i],i)
    s_times=s_times+1

1  m_1263
2017-10-31 13:45:53.054396
process Time
process geo
process wifi
0.999892507793
2017-10-31 13:46:56.500122
2  m_3517
2017-10-31 13:46:56.532887
process Time
process geo
process wifi
0.9999099991
2017-10-31 13:47:57.066234
3  m_5892
2017-10-31 13:47:57.092849
process Time
process geo
process wifi
0.999944236882
2017-10-31 13:49:32.282869
4  m_3054
2017-10-31 13:49:32.317853
process Time
process geo
process wifi
0.999913247159
2017-10-31 13:50:30.264300
5  m_4168
2017-10-31 13:50:30.285822
process Time
process geo
process wifi
1.0
2017-10-31 13:51:11.500175
6  m_4406
2017-10-31 13:51:11.522157
process Time
process geo
process wifi
1.0
2017-10-31 13:51:49.777647
7  m_2878
2017-10-31 13:51:49.795020
process Time
process geo
process wifi
0.9999031477
2017-10-31 13:52:50.480024
8  m_3112
2017-10-31 13:52:50.505804
process Time
process geo
process wifi
1.0
2017-10-31 13:53:37.300082
9  m_625
2017-10-31 13:53:37.326822
process Time
process geo
process wifi
1.0
2017-10-31 13:54:32.448

2017-10-31 15:04:54.361409
75  m_4187
2017-10-31 15:04:54.381934
process Time
process geo
process wifi
0.999264705882
2017-10-31 15:05:57.946597
76  m_6803
2017-10-31 15:05:57.969532
process Time
process geo
process wifi
1.0
2017-10-31 15:06:21.104842
77  m_1089
2017-10-31 15:06:21.120591
process Time
process geo
process wifi
1.0
2017-10-31 15:06:56.796212
78  m_2715
2017-10-31 15:06:56.820900
process Time
process geo
process wifi
0.999813014211
2017-10-31 15:07:32.075613
79  m_5352
2017-10-31 15:07:32.097092
process Time
process geo
process wifi
1.0
2017-10-31 15:08:21.179399
80  m_4548
2017-10-31 15:08:21.200929
process Time
process geo
process wifi
1.0
2017-10-31 15:08:53.343108
81  m_1831
2017-10-31 15:08:53.359806
process Time
process geo
process wifi
1.0
2017-10-31 15:09:35.636954
82  m_3839
2017-10-31 15:09:35.656217
process Time
process geo
process wifi
0.998412208354
2017-10-31 15:11:22.311383
83  m_3871
2017-10-31 15:11:22.345437
process Time
process geo
process wifi
0.999939

In [ ]:
for i in mall_data.keys():
    generateData11(mall_data[i],i)
    s_times=s_times+1

1  m_1263
2017-10-31 16:13:36.692256
process Time
process geo
process wifi
2  m_3517
2017-10-31 16:13:55.993999
process Time
process geo
process wifi
3  m_5892
2017-10-31 16:14:13.482083
process Time
process geo
process wifi
4  m_3054
2017-10-31 16:14:34.843041
process Time
process geo
process wifi
5  m_4168
2017-10-31 16:14:51.602868
process Time
process geo
process wifi
6  m_4406
2017-10-31 16:15:06.625701
process Time
process geo
process wifi
7  m_2878
2017-10-31 16:15:25.000955
process Time
process geo
process wifi
8  m_3112
2017-10-31 16:15:41.111075
process Time
process geo
process wifi
9  m_625
2017-10-31 16:15:57.016594
process Time
process geo
process wifi
10  m_626
2017-10-31 16:16:14.475357
process Time
process geo
process wifi
11  m_623
2017-10-31 16:16:29.770249
process Time
process geo
process wifi
12  m_622
2017-10-31 16:16:43.852542
process Time
process geo
process wifi
13  m_4011
2017-10-31 16:17:00.161905
process Time
process geo
process wifi
14  m_3445
2017-10-31 16:

,user_id,shop_id,longitude,latitude,count,connect,bss0,sig0,bss1,sig1,...,geohash_97,geohash_98,geohash_101,geohash_102,geohash_103,geohash_104,geohash_105,geohash_106,geohash_107,geohash_108
432300,0,s_3850197,121.532312,38.215055,10,0,3022,-59,1673,-59,...,2033,2043,10197,10409,10404,10194,10406,10408,10198,10410
1050775,0,s_160952,121.531050,38.215790,10,0,1196,-68,175,-69,...,1756,1760,9090,9098,9095,9089,9097,9107,9101,9109
1125728,0,s_3985594,121.530301,38.215209,10,0,1074,-31,1083,-45,...,591,595,1548,1556,1549,1547,1555,1572,1570,1578
749210,0,s_501517,121.531628,38.216244,10,0,2535,-42,333,-60,...,2453,2465,13056,13202,13158,13037,13160,13201,13057,13203
660674,0,s_3558111,121.530587,38.215525,10,1240,2351,-34,2338,-34,...,777,789,3275,3476,3274,3272,3473,3278,3276,3477
287298,0,s_454698,121.531582,38.216245,10,0,1841,-38,2125,-54,...,2452,2464,13041,13048,13027,13005,13029,13047,13042,13049
666097,0,s_569106,121.532000,38.216444,10,0,2535,-68,797,-74,...,2545,2556,14968,14980,14969,14967,14979,14973,14971,14981
1028738,0,s_498699,121.531792,38.216397,10,0,2409,-46,2535,-46,...,2479,2522,14097,14653,14640,14086,14642,14652,14098,14654
192696,0,s_2887645,121.532379,38.214972,10,0,1650,-49,1019,-58,...,2020,2024,10271,10279,10276,10270,10278,10288,10282,10290
573864,0,s_3613866,121.530443,38.214805,9,1352,5,-32,6,-56,...,479,515,953,961,958,952,960,969,963,971


In [21]:
fe=[x*1.0/su for x in fea_imp]

In [25]:
train=generateData11(mall_data['m_1263'],'m_1263')

98  m_1263
2017-10-31 13:23:29.983508
process Time
process geo
process wifi


In [26]:
tr=train.columns.tolist()

In [27]:
featu=zip(tr,fe)

In [28]:
featu

[('user_id', 0.010663157351499201),
 ('longitude', 0.065267413256266019),
 ('latitude', 0.059812721840899578),
 ('count', 0.0006456681174129696),
 ('connect', 0.011692556467289398),
 ('bss0', 0.10466314588314898),
 ('sig0', 0.050005447466354727),
 ('bss1', 0.063293595499819374),
 ('sig1', 0.031260773081488363),
 ('bss2', 0.049969780897169039),
 ('sig2', 0.022226465511803798),
 ('bss3', 0.043467341005659632),
 ('sig3', 0.017588320632135463),
 ('bss4', 0.039941167363368946),
 ('sig4', 0.015168842786121002),
 ('bss5', 0.037282459731755285),
 ('sig5', 0.013915581474027054),
 ('bss6', 0.035663243364125852),
 ('sig6', 0.013286313097429369),
 ('bss7', 0.034281192479055929),
 ('sig7', 0.013381729771263754),
 ('bss8', 0.033449393037564584),
 ('sig8', 0.014420532934234745),
 ('bss9', 0.033499968462036893),
 ('sig9', 0.017960927330798827),
 ('day_of_week', 0.013605821334571916),
 ('day_of_month', 0.032789504165878218),
 ('hour', 0.024303957154243577),
 ('minute', 0.011722947595373668),
 ('is_week

In [13]:
gcv,train=generateData11(mall_data['m_1263'],'m_1263')

1  m_1263
2017-10-31 09:39:16.771860
process Time
process geo
process wifi


In [22]:
l=gcv.feature_importances_
k=train.columns.tolist()
kk=zip(k,l)
kk=sorted(kk,key=lambda x:x[1],reverse=True)

In [25]:
fea_imp=[0  for i in range(0,47)]

In [28]:
fea_imp=fea_imp+gcv.feature_importances_

In [29]:
fea_imp

array([ 1764, 11500,  9834,   110,  1862, 14972,  9976,  9850,  6358,
        8822,  4238,  8514,  3184,  8130,  3088,  7532,  2888,  6488,
        2848,  6878,  2382,  6886,  2976,  7480,  3800,  3174,  6352,
        4660,  2744,    36,     0,    16,    18,   350,  2142,  1898,
        1936,  1776,     0,   714,   436,   502,   508,   426,   434,
         868,   568])

In [24]:
len(kk)

47

In [11]:
gcv.best_score_

0.8336020638503708

In [14]:
gcv.best_score_

0.8310222508868107

In [ ]:
for i in mall_data.keys():
    generateData11(mall_data[i],i)
    s_times=s_times+1

In [ ]:
for i in mall_data.keys():
    generateData11(mall_data[i],i)
    s_times=s_times+1

In [ ]:
test[test.user_id!=0].count()

In [ ]:
train.columns

In [ ]:
test.columns

In [ ]:
clf.best_score_

In [ ]:
clf.best_params_

In [ ]:
l=clf.feature_importances_
su=l.sum()

In [ ]:
k=[x*1.0/su for x in l]

In [ ]:
k

In [ ]:
train.columns

In [ ]:
test.columns

In [ ]:
train

In [ ]:
print datetime.datetime.now()
train=proceeWifi(train,0)
print datetime.datetime.now()

In [ ]:
for i in range(4):
    print i

In [ ]:
import joblib